# This notebook is for testing and Integration of fields_building_numerical_all.py with the develop branch

In [1]:
import os
import time
import functools
from shutil import copy
import matplotlib.pyplot as plt
from astropy.io import fits
import astropy.units as u
import hcipy as hc
import numpy as np
os.chdir('/Users/asahoo/repos/PASTIS/') #c
import pastis.util as util     #c
from pastis.config import CONFIG_PASTIS #c
from pastis.e2e_simulators.luvoir_imaging import LuvoirA_APLC #c

In [2]:
root_dir = CONFIG_PASTIS.get('local', 'local_data_path') #c
output_dir = CONFIG_PASTIS.get('local', 'output_data_folder') #needs to be added to config_ini
coronagraph_design = 'small' #c user provides
overall_dir = output_dir + 'luvoirA-'+ coronagraph_design #c
resDir = os.path.join(overall_dir, 'matrix_numerical')

In [3]:
max_LO = CONFIG_PASTIS.getint('calibration', 'max_LO') #c
max_MID = CONFIG_PASTIS.getint('calibration', 'max_MID') #c
max_HI = CONFIG_PASTIS.getint('calibration', 'max_HI') #c
num_DM_act = CONFIG_PASTIS.getint('calibration', 'num_DM_act') #c

In [4]:
nb_seg = CONFIG_PASTIS.getint('LUVOIR', 'nb_subapertures')
wvln = CONFIG_PASTIS.getfloat('LUVOIR', 'lambda') * 1e-9  # m
diam = CONFIG_PASTIS.getfloat('LUVOIR', 'diameter')  # m
nm_aber = CONFIG_PASTIS.getfloat('calibration', 'single_aberration') * 1e-9   # m

In [5]:
im_lamD = CONFIG_PASTIS.getfloat('numerical', 'im_size_lamD_hcipy')  # image size in lambda/D
sampling = CONFIG_PASTIS.getfloat('numerical', 'sampling')

In [6]:
print('LUVOIR apodizer design: {}'.format(coronagraph_design)) #c
print()
print('Wavelength: {} m'.format(wvln))
print('Telescope diameter: {} m'.format(diam))
print('Number of segments: {}'.format(nb_seg))
print()
print('Image size: {} lambda/D'.format(im_lamD))
print('Sampling: {} px per lambda/D'.format(sampling))

LUVOIR apodizer design: small

Wavelength: 5.000000000000001e-07 m
Telescope diameter: 15.0 m
Number of segments: 120

Image size: 30.0 lambda/D
Sampling: 4.0 px per lambda/D


In [7]:
os.makedirs(resDir, exist_ok=True)
os.makedirs(os.path.join(resDir, 'OTE_images'), exist_ok=True)
os.makedirs(os.path.join(resDir, 'psfs'), exist_ok=True)

In [8]:
util.copy_config(resDir)

Saving the configfile to outputs folder.


In [9]:
optics_input = CONFIG_PASTIS.get('LUVOIR', 'optics_path') #c
luvoir = LuvoirA_APLC(optics_input, coronagraph_design, sampling) #c

/opt/miniconda3/envs/pastis_global/lib/python3.7/site-packages/hcipy/field/cartesian_grid.py:74: UserWarning: No automatic weights could be calculated for this grid.
  self.weights *= np.abs(scale)**self.ndim


In [10]:
luvoir.create_global_zernike_mirror(max_LO)
luvoir.create_segmented_mirror(max_MID)
luvoir.create_ripple_mirror(max_HI)
luvoir.create_continuous_deformable_mirror(num_DM_act)

In [11]:
n_LO = luvoir.zernike_mirror.num_actuators #c
n_MID = luvoir.sm.num_actuators 
n_HI = luvoir.ripple_mirror.num_actuators #c
n_DM = luvoir.dm.num_actuators

In [12]:
z_pup_downsample = CONFIG_PASTIS.getfloat('numerical', 'z_pup_downsample') #c
N_pup_z = np.int(luvoir.pupil_grid.shape[0] / z_pup_downsample)
grid_zernike = hc.field.make_pupil_grid(N_pup_z, diameter=luvoir.diam)

In [13]:
dh_outer = hc.circular_aperture(2 * luvoir.apod_dict[coronagraph_design]['owa'] * luvoir.lam_over_d)(luvoir.focal_det) #c
dh_inner = hc.circular_aperture(2 * luvoir.apod_dict[coronagraph_design]['iwa'] * luvoir.lam_over_d)(luvoir.focal_det) #c
dh_mask = (dh_outer - dh_inner).astype('bool')

In [14]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)
DM_modes = np.zeros(n_DM)

In [15]:
luvoir.zernike_mirror.actuators = LO_modes #c
luvoir.sm.actuators = MID_modes
luvoir.ripple_mirror.actuators = HI_modes #c
luvoir.dm.actuators = DM_modes

In [16]:
unaberrated_coro_psf, ref = luvoir.calc_psf(ref=True, display_intermediate=False, return_intermediate=None, norm_one_photon=False)

In [17]:
norm = np.max(ref)

In [18]:
dh_intensity = (unaberrated_coro_psf / norm) * dh_mask
contrast_floor = np.mean(dh_intensity[np.where(dh_mask != 0)])
print('contrast floor: {}'.format(contrast_floor))

contrast floor: 4.237636070056564e-11


In [19]:
nonaberrated_coro_psf, ref,inter_ref = luvoir.calc_psf(ref=True, display_intermediate=False, return_intermediate='efield')
#Efield_ref = inter_ref['at_science_focus'].electric_field

In [20]:
Efield_ref = inter_ref['at_science_focus'].electric_field

In [21]:
print('Generating the Efield for LO modes to science plane')
print('number of LO modes'.format(n_LO))

Generating the Efield for LO modes to science plane
number of LO modes


In [22]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)
DM_modes = np.zeros(n_DM)

In [23]:
print('nm_aber: {} m'.format(nm_aber))
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

nm_aber: 1e-09 m


In [25]:
for pp in range(0,n_LO):
    print(pp)
    LO_modes = np.zeros(n_LO)
    LO_modes[pp] = nm_aber / 2
    luvoir.zernike_mirror.actuators  = LO_modes
    aberrated_coro_psf, ref, inter = luvoir.calc_psf(ref=True, display_intermediate=False, return_intermediate='efield')
    focus_field1 = inter['at_science_focus']
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [26]:
mat_fast = np.zeros([n_LO, n_LO])
for i in range(0, n_LO):
    for j in range(0, n_LO):
        test = np.real((focus_fieldS[i].electric_field - Efield_ref) * np.conj(focus_fieldS[j].electric_field - Efield_ref))
        dh_test = (test / norm) * dh_mask
        contrast = np.mean(dh_test[np.where(dh_mask != 0)])
        mat_fast[i, j] = contrast

In [27]:
matrix_pastis = np.copy(mat_fast) 

In [28]:
matrix_pastis /= np.square(nm_aber * 1e9)

In [29]:
filename_matrix = 'PASTISmatrix_num_LO_' + str(max_LO)
hc.write_fits(matrix_pastis, os.path.join(resDir, filename_matrix + '.fits'))
print('Matrix saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Matrix saved to: /Users/asahoo/Desktop/data_repos/laurent_dataluvoirA-small/matrix_numerical/PASTISmatrix_num_LO_15.fits


In [30]:
filename_matrix = 'EFIELD_Re_matrix_num_LO_' + str(max_LO)
hc.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Efield Real saved to: /Users/asahoo/Desktop/data_repos/laurent_dataluvoirA-small/matrix_numerical/EFIELD_Re_matrix_num_LO_15.fits


In [31]:
filename_matrix = 'EFIELD_Im_matrix_num_LO_' + str(max_LO)
hc.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Efield Imag saved to: /Users/asahoo/Desktop/data_repos/laurent_dataluvoirA-small/matrix_numerical/EFIELD_Im_matrix_num_LO_15.fits


In [32]:
end_time = time.time()

In [33]:
print('Runtime for LO modes:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

Runtime for LO modes: 404.1262400150299 sec = 6.735437333583832 min
Data saved to /Users/asahoo/Desktop/data_repos/laurent_dataluvoirA-small/matrix_numerical


In [34]:
print('Generating the Efield for MID modes to science plane')
print('number of MID modes'.format(n_MID))

Generating the Efield for MID modes to science plane
number of MID modes


In [35]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)
DM_modes = np.zeros(n_DM)

In [36]:
print('nm_aber: {} m'.format(nm_aber))
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

nm_aber: 1e-09 m


In [37]:
for pp in range(0,n_MID):
    print(pp)
    MID_modes = np.zeros(n_MID)
    MID_modes[pp] = nm_aber / 2
    luvoir.sm.actuators  = MID_modes
    aberrated_coro_psf, ref, inter = luvoir.calc_psf(ref=True, display_intermediate=False, return_intermediate='efield')
    focus_field1 = inter['at_science_focus']
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [38]:
mat_fast = np.zeros([n_MID, n_MID])
for i in range(0, n_MID):
    for j in range(0, n_MID):
        test = np.real((focus_fieldS[i].electric_field - Efield_ref) * np.conj(focus_fieldS[j].electric_field - Efield_ref))
        dh_test = (test / norm) * dh_mask
        contrast = np.mean(dh_test[np.where(dh_mask != 0)])
        mat_fast[i, j] = contrast

In [39]:
matrix_pastis = np.copy(mat_fast)
matrix_pastis /= np.square(nm_aber * 1e9)  

In [40]:
filename_matrix = 'PASTISmatrix_num_MID_' + str(max_MID)
hc.write_fits(matrix_pastis, os.path.join(resDir, filename_matrix + '.fits'))
print('Matrix saved to:', os.path.join(resDir, filename_matrix + '.fits'))


Matrix saved to: /Users/asahoo/Desktop/data_repos/laurent_dataluvoirA-small/matrix_numerical/PASTISmatrix_num_MID_6.fits


In [41]:
filename_matrix = 'EFIELD_Re_matrix_num_MID_' + str(max_MID)
hc.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Efield Real saved to: /Users/asahoo/Desktop/data_repos/laurent_dataluvoirA-small/matrix_numerical/EFIELD_Re_matrix_num_MID_6.fits


In [42]:
filename_matrix = 'EFIELD_Im_matrix_num_MID_' + str(max_MID)
hc.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))


Efield Imag saved to: /Users/asahoo/Desktop/data_repos/laurent_dataluvoirA-small/matrix_numerical/EFIELD_Im_matrix_num_MID_6.fits


In [43]:
end_time = time.time()
print('Runtime for MID modes:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

Runtime for MID modes: 6250.768197059631 sec = 104.17946995099386 min
Data saved to /Users/asahoo/Desktop/data_repos/laurent_dataluvoirA-small/matrix_numerical


In [44]:
print('Generating the Efield for HI modes to science plane')
print('number of HI modes'.format(n_HI))

Generating the Efield for HI modes to science plane
number of HI modes


In [45]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)
DM_modes = np.zeros(n_DM)

In [46]:
print('nm_aber: {} m'.format(nm_aber))
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

nm_aber: 1e-09 m


In [47]:
for pp in range(0, n_HI):
    print(pp)
    HI_modes = np.zeros(n_HI)
    HI_modes[pp] = nm_aber / 2
    luvoir.ripple_mirror.actuators = HI_modes
    aberrated_coro_psf, ref, inter = luvoir.calc_psf(ref=True, display_intermediate=False, return_intermediate='efield')
    focus_field1 = inter['at_science_focus']
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [48]:
mat_fast = np.zeros([n_HI, n_HI])
for i in range(0, n_HI):
    for j in range(0, n_HI):
        test = np.real((focus_fieldS[i].electric_field - Efield_ref) * np.conj(focus_fieldS[j].electric_field - Efield_ref))
        dh_test = (test / norm) * dh_mask
        contrast = np.mean(dh_test[np.where(dh_mask != 0)])
        mat_fast[i, j] = contrast

In [49]:
matrix_pastis = np.copy(mat_fast) 
matrix_pastis /= np.square(nm_aber * 1e9)

In [50]:
filename_matrix = 'PASTISmatrix_num_HI_' + str(max_HI)
hc.write_fits(matrix_pastis, os.path.join(resDir, filename_matrix + '.fits'))
print('Matrix saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Matrix saved to: /Users/asahoo/Desktop/data_repos/laurent_dataluvoirA-small/matrix_numerical/PASTISmatrix_num_HI_24.fits


In [51]:
filename_matrix = 'EFIELD_Re_matrix_num_HI_' + str(max_HI)
hc.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Efield Real saved to: /Users/asahoo/Desktop/data_repos/laurent_dataluvoirA-small/matrix_numerical/EFIELD_Re_matrix_num_HI_24.fits


In [52]:
filename_matrix = 'EFIELD_Im_matrix_num_HI_' + str(max_HI)
hc.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Efield Imag saved to: /Users/asahoo/Desktop/data_repos/laurent_dataluvoirA-small/matrix_numerical/EFIELD_Im_matrix_num_HI_24.fits


In [53]:
end_time = time.time()
print('Runtime for HI modes:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

Runtime for HI modes: 1793.4154500961304 sec = 29.89025750160217 min
Data saved to /Users/asahoo/Desktop/data_repos/laurent_dataluvoirA-small/matrix_numerical


In [54]:
print('Generating the Efield for LO modes to LOWFS')
print('number of LO modes'.format(n_LO))

Generating the Efield for LO modes to LOWFS
number of LO modes


In [55]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)
DM_modes = np.zeros(n_DM)

In [56]:
luvoir.zernike_mirror.actuators = LO_modes #c
luvoir.sm.actuators = MID_modes
luvoir.ripple_mirror.actuators = HI_modes #c
luvoir.dm.actuators = DM_modes

In [61]:
zernike_ref = luvoir.calc_low_order_wfs()

In [63]:
zernike_ref_sub_real = hc.field.subsample_field(zernike_ref.real, z_pup_downsample, grid_zernike, statistic='mean')
zernike_ref_sub_imag = hc.field.subsample_field(zernike_ref.imag, z_pup_downsample, grid_zernike, statistic='mean')
Efield_ref = zernike_ref_sub_real + 1j*zernike_ref_sub_imag

In [64]:
print('nm_aber: {} m'.format(nm_aber))
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

nm_aber: 1e-09 m


In [65]:
for pp in range(0, n_LO):
    print(pp)
    LO_modes = np.zeros(n_LO)
    LO_modes[pp] = nm_aber / 2
    luvoir.zernike_mirror.actuators = LO_modes
    zernike_meas = luvoir.calc_low_order_wfs()
    zernike_meas_sub_real = hc.field.subsample_field(zernike_meas.real, z_pup_downsample, grid_zernike,statistic='mean')
    zernike_meas_sub_imag = hc.field.subsample_field(zernike_meas.imag, z_pup_downsample, grid_zernike,statistic='mean')
    focus_field1 = zernike_meas_sub_real + 1j * zernike_meas_sub_imag
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [66]:
filename_matrix = 'EFIELD_LOWFS_Re_matrix_num_LO_' + str(max_LO)
hc.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Efield Real saved to: /Users/asahoo/Desktop/data_repos/laurent_dataluvoirA-small/matrix_numerical/EFIELD_LOWFS_Re_matrix_num_LO_15.fits


In [67]:
filename_matrix = 'EFIELD_LOWFS_Im_matrix_num_LO_' + str(max_LO)
hc.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Efield Imag saved to: /Users/asahoo/Desktop/data_repos/laurent_dataluvoirA-small/matrix_numerical/EFIELD_LOWFS_Im_matrix_num_LO_15.fits


In [68]:
end_time = time.time()
print('Runtime for LO modes and LOWFS:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

Runtime for LO modes and LOWFS: 232.77843189239502 sec = 3.879640531539917 min
Data saved to /Users/asahoo/Desktop/data_repos/laurent_dataluvoirA-small/matrix_numerical


In [69]:
print('Generating the Efield for MID modes to OBWFS')
print('number of MID modes'.format(n_MID))

Generating the Efield for MID modes to OBWFS
number of MID modes


In [71]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)
DM_modes = np.zeros(n_DM)
luvoir.zernike_mirror.actuators = LO_modes
luvoir.sm.actuators = MID_modes
luvoir.ripple_mirror.actuators = HI_modes
luvoir.dm.actuators = DM_modes

In [72]:
zernike_ref = luvoir.calc_out_of_band_wfs()
zernike_ref_sub_real = hc.field.subsample_field(zernike_ref.real, z_pup_downsample, grid_zernike, statistic='mean')
zernike_ref_sub_imag = hc.field.subsample_field(zernike_ref.imag, z_pup_downsample, grid_zernike, statistic='mean')
Efield_ref = zernike_ref_sub_real + 1j*zernike_ref_sub_imag

In [73]:
print('nm_aber: {} m'.format(nm_aber))
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

nm_aber: 1e-09 m


In [74]:
for pp in range(0, n_MID):
    print(pp)
    MID_modes = np.zeros(n_MID)
    MID_modes[pp] = nm_aber / 2
    luvoir.sm.actuators = MID_modes
    zernike_meas = luvoir.calc_out_of_band_wfs()
    zernike_meas_sub_real = hc.field.subsample_field(zernike_meas.real, z_pup_downsample, grid_zernike, statistic='mean')
    zernike_meas_sub_imag = hc.field.subsample_field(zernike_meas.imag, z_pup_downsample, grid_zernike, statistic='mean')
    focus_field1 = zernike_meas_sub_real + 1j * zernike_meas_sub_imag
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [75]:
filename_matrix = 'EFIELD_OBWFS_Re_matrix_num_MID_' + str(max_MID)
hc.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Efield Real saved to: /Users/asahoo/Desktop/data_repos/laurent_dataluvoirA-small/matrix_numerical/EFIELD_OBWFS_Re_matrix_num_MID_6.fits


In [76]:
filename_matrix = 'EFIELD_OBWFS_Im_matrix_num_MID_' + str(max_MID)
hc.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Efield Imag saved to: /Users/asahoo/Desktop/data_repos/laurent_dataluvoirA-small/matrix_numerical/EFIELD_OBWFS_Im_matrix_num_MID_6.fits


In [77]:
end_time = time.time()
print('Runtime for MID modes and OBWFS:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

Runtime for MID modes and OBWFS: 1516.5692591667175 sec = 25.276154319445293 min
Data saved to /Users/asahoo/Desktop/data_repos/laurent_dataluvoirA-small/matrix_numerical
